**MLflow setup:**
- Tracking server: Local server.
- Backend store: Sqlite database.
- Artifacts store: Local filesystem.

**Running MLflow server:**
`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`

In [1]:
# IMPORTING MODULES:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from mlflow.tracking import MlflowClient
from mlflow.exceptions import MlflowException

# IGNORING WARNINGS: 
import warnings
warnings.filterwarnings("ignore")

In [2]:
# CONNECTING EXPERIMENTS:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
# INSPECTING EXPERIMENTS:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [4]:
# CREATING EXPERIMENTS AND LOGGING NEW RUN:
mlflow.set_experiment("my-exp-scenario-1")

with mlflow.start_run():
    X, y = load_iris(return_X_y=True)
    
    params = {"C": 0.1, "random_state": 22}
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params).fit(X, y)               # Initializing LR.
    y_pred = lr.predict(X)                                    # Initializing predictions.
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2022/06/11 14:27:29 INFO mlflow.tracking.fluent: Experiment with name 'my-exp-scenario-1' does not exist. Creating a new experiment.


default artifacts URI: './artifacts_local/1/90247ce20ddd440b8644a15359825257/artifacts'


In [5]:
# INSPECTING EXPERIMENTS:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts_local/1', experiment_id='1', lifecycle_stage='active', name='my-exp-scenario-1', tags={}>]

In [6]:
# INTERACTING WITH MODEL REGISTRY:
client = MlflowClient("http://127.0.0.1:5000")
client.list_registered_models()

[]

In [7]:
# INTERACTING WITH MODEL REGISTRY:
run_id = client.list_run_infos(experiment_id="1")[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="iris-classifier"
)

Successfully registered model 'iris-classifier'.
2022/06/11 14:32:05 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654937225395, current_stage='None', description='', last_updated_timestamp=1654937225395, name='iris-classifier', run_id='90247ce20ddd440b8644a15359825257', run_link='', source='./artifacts_local/1/90247ce20ddd440b8644a15359825257/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>